In [62]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from vnstock import *
from sklearn.cluster import DBSCAN

In [42]:
START_DATE = '2013-01-01'
END_DATE = '2023-10-12'
SYSBOL = 'HPG'

In [43]:
# Lấy dữ liệu giá cả cổ phiếu
# data = pd.read_csv('data.csv', index_col='Date')
df = stock_historical_data(SYSBOL, START_DATE, END_DATE, "1D", "stock")
df.set_index('time')
df.head()

,time,open,high,low,close,volume,ticker
0,2013-01-02,1480,1500,1470,1490,541540,HPG
1,2013-01-03,1480,1510,1470,1510,720080,HPG
2,2013-01-04,1510,1580,1500,1560,569620,HPG
3,2013-01-07,1560,1630,1500,1630,926020,HPG
4,2013-01-08,1700,1710,1680,1700,1480180,HPG


In [44]:
# Tính toán giá đóng cửa bình quân
close = df['close'].rolling(window=10).mean()
close = close.fillna(0)
close.head(20)

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9     1626.0
10    1644.0
11    1655.0
12    1661.0
13    1655.0
14    1639.0
15    1623.0
16    1617.0
17    1617.0
18    1620.0
19    1628.0
Name: close, dtype: float64

In [61]:
# Tạo mô hình K-means
model = KMeans(n_clusters=2, random_state=0)

# Huấn luyện mô hình
array = close.to_numpy().reshape(-1,1)
model.fit(array)

# Dự đoán nhóm cho mỗi ngày
labels = model.predict(array)

# Tìm các điểm giao dịch
buy_signals = []
sell_signals = []
for i in range(len(labels)):
  if labels[i] == 1 and labels[i - 1] == 0:
    buy_signals.append(i)
  elif labels[i] == 0 and labels[i - 1] == 1:
    sell_signals.append(i)

# Tính toán lợi nhuận
profit = 0
for i in range(len(buy_signals)):
  profit += close[buy_signals[i]] - close[sell_signals[i]]

print(profit)

# Vẽ chart
fig = go.Figure()
fig.add_trace(go.Line(x=df['time'], y=close, name='Giá đóng cửa'))
fig.add_trace(go.Scatter(x=df['time'][labels == 1], y=close[labels == 1], name='Nhóm tăng'))
fig.add_trace(go.Scatter(x=df['time'][labels == 0], y=close[labels == 0], name='Nhóm giảm'))
# fig.add_trace(go.Scatter(x=df['time'][buy_signals], y=close[buy_signals], name='Mua'))
# fig.add_trace(go.Scatter(x=df['time'][sell_signals], y=close[sell_signals], name='Bán'))
fig.show()

18186.0


/home/hieuhv10/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [70]:
# Tạo mô hình DBSCAN
model = DBSCAN(eps=110, min_samples=10)

# Huấn luyện mô hình
array = close.to_numpy().reshape(-1,1)
model.fit(array)

# Dự đoán nhóm cho mỗi ngày
labels = model.labels_

# Tìm các điểm giao dịch
buy_signals = []
sell_signals = []
for i in range(len(labels)):
  if labels[i] == -1:
    continue
  elif labels[i] == 0:
    buy_signals.append(i)
  elif labels[i] == 1:
    sell_signals.append(i)

# Tính toán lợi nhuận
profit = 0
for i in range(len(buy_signals)):
  profit += close[buy_signals[i]] - close[sell_signals[i]]

print(profit)

# Vẽ chart
fig = go.Figure()
fig.add_trace(go.Line(x=df['time'], y=close, name='Giá đóng cửa'))
fig.add_trace(go.Scatter(x=df['time'][labels == 0], y=close[labels == 0], name='Cluster 0'))
fig.add_trace(go.Scatter(x=df['time'][labels == 1], y=close[labels == 1], name='Cluster 1'))
# fig.add_trace(go.Scatter(x=df['time'][buy_signals], y=close[buy_signals], marker='x', color='green', name='Mua'))
# fig.add_trace(go.Scatter(x=df['time'][sell_signals], y=close[sell_signals], marker='x', color='red', name='Bán'))
fig.show()

-549341.0
